# 1. ID mapping 

your final_master_sheet_clean.xlsx has a unified patient ID column that lines up with your intraday data.
what we’ll do

##### 1. load final_master_sheet_clean.xlsx.
##### 2. build a mapping Code → patientID (from the table you provided).
##### 3. for every visit sheet (Ramadan, Visit 1 … Visit 7), replace/add a column PatientID (Huawei Data).
##### 4. keep the original Code if you like, or drop it once you confirm the mapping is correct.
##### 5. save a new Excel (e.g., final_master_sheet_clean_with_huawei.xlsx).

In [ ]:
import pandas as pd
from pathlib import Path

# input/output
MASTER_PATH = Path("/kaggle/input/static-variables/final_master_sheet_clean.xlsx")
OUT_PATH = Path("/kaggle/working/final_master_sheet_clean_with_huawei.xlsx")

# your mapping dictionary
PATIENT_ID_MAP = {
    "R01": 45, "R02": 46, "R04": 47, "R05": 48, "R06": 49, "R07": 53,
    "R10": 54, "R11": 55, "R12": 57, "R15": 59, "R16": 60, "R17": 61,
    "R20": 63, "R21": 64, "R22": 66, "R23": 67, "R24": 68, "R25": 69,
    "R26": 70, "R27": 71, "R28": 72, "R29": 73, "R30": 74, "R31": 75,
    "R32": 76, "R33": 77, "R34": 78, "R35": 79, "R36": 80, "R37": 81,
    "R39": 82, "R40": 83, "R41": 84, "R42": 85, "R43": 86,
}
id_map_df = pd.DataFrame(list(PATIENT_ID_MAP.items()), columns=["Code", "PatientID (Huawei Data)"])

# load workbook
xls = pd.ExcelFile(MASTER_PATH)
sheets = {s: pd.read_excel(MASTER_PATH, sheet_name=s) for s in xls.sheet_names}

# update each sheet that has "Code"
updated_sheets = {}
for name, df in sheets.items():
    if "Code" in df.columns:
        df = df.merge(id_map_df, on="Code", how="left")
        # optional: drop old Code col and just keep Huawei ID
        # df = df.drop(columns=["Code"])
        updated_sheets[name] = df
    else:
        updated_sheets[name] = df

# save to new Excel
with pd.ExcelWriter(OUT_PATH, engine="openpyxl") as writer:
    for name, df in updated_sheets.items():
        df.to_excel(writer, index=False, sheet_name=name)

print(f"saved → {OUT_PATH}")


# 2. subperiods + Ramadan/Shawwal periods

Sets main periods (Ramadan/Shawwal) with your exact dates.

Defines visit subperiods as inclusive date ranges:

V1 = 2023-03-13 → 2023-03-26
V2 = 2023-03-27 → 2023-04-02
V3 = 2023-04-03 → 2023-04-09
V4 = 2023-04-10 → 2023-04-19 (ends at Ramadan end)
V5 = 2023-04-20 → 2023-04-26
V6 = 2023-04-27 → 2023-05-08
V7 = 2023-05-09 → 2023-05-19 (ends at Shawwal end)


Add explicit Visit subperiods + Ramadan/Shawwal periods:
- Annotates intraday rows with `period_main` and `visit_assigned`
- Writes spec sheets into master workbook
- Saves:
  /kaggle/working/intraday_with_visits.csv
  /kaggle/working/final_master_sheet_clean_with_visits.xlsx



In [ ]:
# -*- coding: utf-8 -*-
"""
Annotate intraday with Visit subperiods + Ramadan/Shawwal periods
- Adds `visit_assigned` (inclusive visit windows) and `period_main` (Ramadan/Shawwal/outside)
- Saves annotated intraday to CSV and **single-sheet Excel (Intraday_All)**
- Injects spec sheets into the master workbook and adds Visit_Anchor_Date to visit sheets
- Also writes a dynamic missingness-by-visit CSV (optional QA artifact)
"""

import pandas as pd
import numpy as np
from pathlib import Path

# ---------------- CONFIG ----------------
INTRADAY_CSV_PATH = Path("/kaggle/input/intraday/intraday.csv")
MASTER_XLSX_IN    = Path("/kaggle/working/final_master_sheet_clean_with_huawei.xlsx")
MASTER_XLSX_OUT   = Path("/kaggle/working/final_master_sheet_clean_with_visits.xlsx")

INTRADAY_OUT_CSV  = Path("/kaggle/working/intraday_with_visits.csv")
INTRADAY_OUT_XLSX = Path("/kaggle/working/intraday_with_visits.xlsx")

DYN_VISIT_OUT     = Path("/kaggle/working/missingness_dynamic_by_visit.csv")  # optional QA

# Main periods (inclusive)
I_RAMADAN_START = pd.Timestamp("2023-03-22")
I_RAMADAN_END   = pd.Timestamp("2023-04-19")
I_SHAWWAL_START = pd.Timestamp("2023-04-20")
I_SHAWWAL_END   = pd.Timestamp("2023-05-19")

# Visit subperiods (ALL INCLUSIVE — exactly as specified)
VISIT_SUBPERIODS = [
    {"Visit": "Visit 1", "start": "2023-03-13", "end": "2023-03-21"},
    {"Visit": "Visit 2", "start": "2023-03-22", "end": "2023-03-30"},
    {"Visit": "Visit 3", "start": "2023-03-31", "end": "2023-04-06"},
    {"Visit": "Visit 4", "start": "2023-04-08", "end": "2023-04-16"},
    {"Visit": "Visit 5", "start": "2023-04-17", "end": "2023-04-26"},
    {"Visit": "Visit 6", "start": "2023-04-27", "end": "2023-05-08"},
    {"Visit": "Visit 7", "start": "2023-05-09", "end": "2023-05-19"},
]

# Sheets in master we do not modify
SKIP_SHEETS = {"HMC_map_patientID"}


# ---------------- HELPERS ----------------
def _normalize_date_col(s: pd.Series) -> pd.Series:
    return pd.to_datetime(s, errors="coerce").dt.normalize()

def build_visit_spec_df(rows: list[dict]) -> pd.DataFrame:
    """
    Build inclusive visit windows as a tidy DataFrame, warn (don't crash) on overlaps or bad spans.
    """
    df = pd.DataFrame(rows)
    df["start"] = _normalize_date_col(df["start"])
    df["end"]   = _normalize_date_col(df["end"])
    df = df.sort_values(["start", "end"]).reset_index(drop=True)

    bad_span = df[df["end"] < df["start"]]
    if not bad_span.empty:
        print("⚠️ Visit with end < start:\n", bad_span)

    df["prev_end"] = df["end"].shift(1)
    overlap = df[(df.index > 0) & (df["start"] <= df["prev_end"])]
    if not overlap.empty:
        print("⚠️ Overlapping visits detected (inclusive ranges). Ensure previous 'end' < next 'start'.")
        print(overlap[["Visit", "start", "end", "prev_end"]])

    return df.drop(columns=["prev_end"], errors="ignore")

def tag_main_period(ts: pd.Timestamp) -> str | float:
    if pd.isna(ts):
        return np.nan
    if I_RAMADAN_START <= ts <= I_RAMADAN_END:
        return "Ramadan (Mar 22–Apr 19, 2023)"
    if I_SHAWWAL_START <= ts <= I_SHAWWAL_END:
        return "Shawwal (Apr 20–May 19, 2023)"
    return "Outside Ramadan/Shawwal"

def assign_visit_inclusive(dates: pd.Series, visit_spec: pd.DataFrame) -> pd.Series:
    """
    Inclusive mapping: a date belongs to Visit i if start_i <= date <= end_i.
    Dates outside all windows remain NaN.
    """
    visits_cat = pd.CategoricalDtype(categories=list(visit_spec["Visit"]), ordered=True)
    out = pd.Series(pd.Categorical([None] * len(dates), dtype=visits_cat), index=dates.index)
    for _, r in visit_spec.iterrows():
        mask = (dates >= r["start"]) & (dates <= r["end"])
        out.loc[mask] = r["Visit"]
    return out

def _order_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Put key columns first for readability if they exist."""
    cols = list(df.columns)
    front = [c for c in ["date", "visit_assigned", "period_main", "patientID", "huaweiID"] if c in cols]
    rest  = [c for c in cols if c not in front]
    return df[front + rest]


# ---------------- 1) LOAD & ANNOTATE INTRADAY ----------------
print("Annotating intraday with visit subperiods & main periods…")
intraday = pd.read_csv(INTRADAY_CSV_PATH)
if "date" not in intraday.columns:
    raise ValueError("intraday.csv must have a 'date' column")

intraday["date"] = _normalize_date_col(intraday["date"])

visit_spec_df = build_visit_spec_df(VISIT_SUBPERIODS)
intraday["period_main"]    = intraday["date"].apply(tag_main_period)
intraday["visit_assigned"] = assign_visit_inclusive(intraday["date"], visit_spec_df)

# Save CSV once
intraday.to_csv(INTRADAY_OUT_CSV, index=False)
print(f"✅ Saved annotated intraday CSV → {INTRADAY_OUT_CSV}")

# ---------------- 2) UPDATE MASTER WORKBOOK ----------------
print("Injecting Visit_Subperiods_Spec & Period_Bounds into master…")
xls = pd.ExcelFile(MASTER_XLSX_IN)
sheets = {s: pd.read_excel(MASTER_XLSX_IN, sheet_name=s) for s in xls.sheet_names}

# optional: add a Visit_Anchor_Date to each Visit sheet (start of its inclusive subperiod)
ANCHORS = {row["Visit"]: row["start"] for _, row in visit_spec_df.iterrows()}

updated = {}
for name, df in sheets.items():
    if name in SKIP_SHEETS:
        updated[name] = df
        continue
    if name in ANCHORS and "Visit_Anchor_Date" not in df.columns:
        df = df.copy()
        df["Visit_Anchor_Date"] = ANCHORS[name].date()
    updated[name] = df

period_bounds = pd.DataFrame({
    "Period":    ["Ramadan", "Shawwal"],
    "Start":     [I_RAMADAN_START.date(), I_SHAWWAL_START.date()],
    "End":       [I_RAMADAN_END.date(),   I_SHAWWAL_END.date()],
    "Inclusive": [True, True],
})

with pd.ExcelWriter(MASTER_XLSX_OUT, engine="openpyxl") as writer:
    for name, df in updated.items():
        df.to_excel(writer, index=False, sheet_name=name[:31])  # safe truncate
    visit_spec_df.to_excel(writer, index=False, sheet_name="Visit_Subperiods_Spec")
    period_bounds.to_excel(writer, index=False, sheet_name="Period_Bounds")

print(f"✅ Saved master with specs → {MASTER_XLSX_OUT}")

# ---------------- 3) OPTIONAL QA: MISSINGNESS BY VISIT (CSV) ----------------
EXCLUDE = {"Unnamed: 0", "huaweiID", "date", "start", "period_main", "visit_assigned"}
dfv = intraday.copy()
feature_cols = [c for c in dfv.columns if c not in EXCLUDE and c != "patientID"]

if feature_cols:
    out = (
        dfv.groupby("visit_assigned")[feature_cols]
           .apply(lambda g: g.isna().mean().mul(100).round(2))
           .reset_index()
           .rename(columns={"visit_assigned": "Visit"})
    )
    out.melt(id_vars=["Visit"], var_name="Feature", value_name="% Missing (Visit Window)") \
       .to_csv(DYN_VISIT_OUT, index=False)
    print(f"✅ Saved dynamic-by-visit missingness CSV → {DYN_VISIT_OUT}")
else:
    print("ℹ️ Skipped missingness export (no feature columns found).")

# ---------------- 4) SINGLE-SHEET EXCEL EXPORT ----------------
print("Writing single-sheet annotated intraday Excel…")
intraday_ordered = _order_columns(intraday)

with pd.ExcelWriter(INTRADAY_OUT_XLSX, engine="openpyxl") as writer:
    intraday_ordered.to_excel(writer, index=False, sheet_name="Intraday_All")

print(f"✅ Saved single-sheet intraday Excel → {INTRADAY_OUT_XLSX}")

# ---------------- 5) QUICK CHECKS (stdout) ----------------
print("\n— Quick checks —")
try:
    print("Date coverage:", intraday["date"].min(), "→", intraday["date"].max(), "| rows:", len(intraday))
    print("\nVisit assignment counts:")
    print(intraday["visit_assigned"].value_counts(dropna=False))
    print("\nMain period counts:")
    print(intraday["period_main"].value_counts(dropna=False))
except Exception as e:
    print("Sanity checks skipped:", e)


# 3. Comprehensive data-quality and missingness analysis pipeline

Unified Missingness Pipeline (Qatar edition, percent-aware)
- Normalized name alignment (cleaned + aliases + percent-aware)
- Separate Static vs Dynamic outputs
- Ramadan/Shawwal rollups using visit groups
- QC conditional formatting (>30 warn, >50 alert)
- Patient-level missingness per visit
- Robust dictionary merge (canonical-first, strict percent-safe fallback)

This Python script is a **comprehensive data-quality and missingness analysis pipeline** tailored for a multi-sheet Excel dataset (like your *final_master_sheet_clean_with_huawei.xlsx*) used in clinical research (e.g., your Ramadan–Shawwal T1D study). Below is a full explanation of its structure and logic:

---

## 🧩 1. Purpose

It automatically:

* Reads **multiple Excel sheets** (each representing a study visit or phase),
* Calculates **missing-data statistics** per feature, visit, and patient,
* Optionally merges a **data dictionary** for metadata categories (e.g., clinical domains),
* Summarizes **missingness by Ramadan vs. Shawwal**, and
* Produces a **color-coded Excel report** and CSVs for static, dynamic (intraday), and patient-level missingness.

---

## 📁 2. Input Files

### Required

* `MASTER_PATH` → Excel file containing all study visits.

  * Each sheet = one visit (e.g., *Visit 2, Visit 3, Visit 6 Shawwal*).
* `PATIENT_ID_COL` → patient identifier column (used for patient-level summaries).

### Optional

* `DICT_PATH` → data dictionary mapping features to categories, units, and definitions.
* `INTRADAY_CSV_PATH` → Huawei intraday file (continuous wearable data).
* `DYN_VISIT_OUT` → optional CSV for dynamic missingness per visit.

---

## 🧠 3. Core Functions

### (a) **String Normalization**

Functions like `_strip_accents`, `clean_name_base`, and `canonical_key` standardize column names:

* Remove accents, punctuation, and spacing.
* Detect if a variable is percentage-based (e.g., `HbA1c (%)` → `hba1c_pct`).
* Build a consistent key system to match feature names even with minor formatting differences.

This ensures reliable merging across sheets and with the data dictionary.

---

### (b) **Missingness Computation**

For each sheet (visit):

* `percent_missing_by_column(df, visit_name)`
  → computes % missing values per feature.
* `missing_counts(df, visit_name)`
  → counts missing and total entries per feature.

These are then merged across all visits using Pandas `reduce()`.

---

### (c) **Weighted Overall Missingness**

`weighted_overall_pct()` combines the visit-level counts to yield an overall % missing per feature, weighting by total entries rather than raw averages (so small visits don’t dominate).

---

### (d) **Dictionary Merge**

If the data dictionary exists:

* Matches each feature to its category, unit, and definition using both:

  1. canonical name (exact match),
  2. fallback on base name + “is percent” flag.
* Enriches the report with metadata columns:

  * `Category`, `Definition & Unit`, `Unit/Type`, `Feature Type`, etc.

This links missingness to clinical domains (e.g., “CGM metrics,” “Lipid profile”).

---

### (e) **Roll-up by Ramadan vs. Shawwal**

Groups visits into `RAMADAN_VISITS` and `SHAWWAL_VISITS`.
Computes combined % missing within each group:

```python
% Missing Ramadan (rollup)
% Missing Shawwal (rollup)
```

— weighted by total cells.

---

### (f) **Patient-Level Missingness**

For each visit:

* Calculates per-patient row missingness across all variables.
* Reports percentage and number of missing cells:

  ```
  Row % Missing (all fields)
  Missing Cells
  Total Cells
  ```

This identifies patients with systematically incomplete records.

---

### (g) **Dynamic (Intraday) Missingness**

If `INCLUDE_INTRADAY=True` and a CSV exists:

* Splits Huawei or CGM data by date ranges:

  * Ramadan: March 23 – April 21 2023
  * Shawwal: April 22 – May 21 2023
* For each feature (e.g., HR, steps, SpO₂):

  * % missing per phase + overall.
* Merges metadata from the dictionary if possible.

---

## 🧾 4. Output Files

| Output                              | Description                                |
| ----------------------------------- | ------------------------------------------ |
| **`missingness_report_full.xlsx`**  | Master Excel with conditional formatting.  |
| **`missingness_static.csv`**        | Visit-level missingness summary.           |
| **`missingness_dynamic.csv`**       | Intraday (Huawei/CGM) missingness summary. |
| **`missingness_patient_level.csv`** | Patient-wise missingness per visit.        |

### Excel Sheets in `OUT_XLSX`

* `Missingness_Static` (main summary, color-coded)
* `Missingness_Dynamic` (if available)
* `Patient_Level_Missingness`
* `Unmapped_Static` / `Unmapped_Dynamic` → features not found in dictionary
* `Name_Mapping_*` → audit of feature–dictionary mappings

Conditional formatting flags:

* Yellow if > 30% missing,
* Red if > 50%.

---

## 📊 5. Key Outputs and Interpretation

| Column                                 | Meaning                                        |
| -------------------------------------- | ---------------------------------------------- |
| `% Missing Visit X`                    | Missing fraction per feature per visit.        |
| `Overall % Missing`                    | Weighted total missingness across all visits.  |
| `% Missing Ramadan / Shawwal (rollup)` | Aggregated phase-specific missingness.         |
| `Availability (Visits)`                | JSON list of visits where feature is present.  |
| `Category`                             | Domain (e.g., “Insulin metrics,” “Lifestyle”). |

---

## ⚙️ 6. Why It’s Useful for Your T1D Ramadan–Shawwal Dataset

This script ensures:

* **Transparent data completeness tracking** across visits.
* **Automated integration** with your clinical variable dictionary.
* **Separate summaries** for:

  * Static features (e.g., HbA1c, BMI),
  * Dynamic wearable metrics (HR, steps, sleep),
  * Patient-level data quality.
* Facilitates decisions like:

  * Which features or visits need imputation,
  * Which patient records to exclude,
  * How much data loss differs between Ramadan and Shawwal.

---

### ✅ In Short

> **This code is a reproducible, phase-aware missing-data audit tool** for clinical and wearable datasets, producing publication-ready Excel reports and per-domain summaries that quantify data completeness across visits, patients, and phases (Ramadan vs. Shawwal).



In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
import re
import unicodedata

# =============== CONFIG ===============
# Inputs
MASTER_PATH = Path("/kaggle/working/final_master_sheet_clean_with_huawei.xlsx")
DICT_PATH   = Path("/kaggle/input/static-variables/Categorized_Data_Dictionary.xlsx")  # xlsx or csv
INTRADAY_CSV_PATH = Path("/kaggle/input/intraday/intraday.csv")  # optional dynamic

# Patient ID column in the master visit sheets (after your mapping step)
PATIENT_ID_COL = "PatientID (Huawei Data)"

# Which sheets to treat as visits (the script will read all except SKIP)
SKIP_SHEETS = {"HMC_map_patientID"}

# Visit groups for rollups (edit as needed)
RAMADAN_VISITS = {"Ramadan", "Visit 1", "Visit 2", "Visit 3", "Visit 4 (whole Ramadan)"}
SHAWWAL_VISITS = {"Visit 6 (Shawal)"}  # add Visit 5 here if it belongs to Shawwal

# Intraday options
INCLUDE_INTRADAY = True
RAMADAN_START = pd.to_datetime("2023-03-23")
RAMADAN_END   = pd.to_datetime("2023-04-21")
SHAWWAL_START = pd.to_datetime("2023-04-22")
SHAWWAL_END   = pd.to_datetime("2023-05-21")
# keep patientID to align; exclude only non-feature fields
INTRADAY_EXCLUDE_COLS = {"Unnamed: 0", "huaweiID", "date", "start"}

# Optional stage-2 CSV that (if present) we add as a sheet
DYN_VISIT_OUT = Path("/kaggle/working/missingness_dynamic_by_visit.csv")

# Outputs
OUT_XLSX = Path("/kaggle/working/missingness_report_full.xlsx")
OUT_CSV_STATIC  = Path("/kaggle/working/missingness_static.csv")
OUT_CSV_DYNAMIC = Path("/kaggle/working/missingness_dynamic.csv")
OUT_CSV_PATIENT = Path("/kaggle/working/missingness_patient_level.csv")

# QC thresholds (%)
WARN_THRES = 30.0
ALERT_THRES = 50.0
# =====================================


# ===== Percent-aware normalization =====
def _strip_accents(s: str) -> str:
    s = unicodedata.normalize("NFKD", s)
    return "".join(ch for ch in s if not unicodedata.combining(ch))

def clean_name_base(s: str) -> str:
    """Base alnum key (lower, accents removed, spaces collapsed, punctuation removed)."""
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return ""
    s = str(s).lower().strip()
    s = _strip_accents(s)
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^a-z0-9]", "", s)
    return s

def has_percent(s: str) -> bool:
    """Detect percent variants like 'HbA1c (%)', 'hba1c%', 'hba1c%%'."""
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return False
    s0 = str(s).lower()
    if "%" in s0:
        return True
    if "(%)" in s0:
        return True
    # If you want the word 'percent' to count:
    # if re.search(r"\bpercent\b", s0): return True
    return False

def canonical_key(raw_name: str) -> str:
    """Percent-safe canonical key: base or base+'_pct'."""
    base = clean_name_base(raw_name)
    return f"{base}_pct" if has_percent(raw_name) else base

# Canonical targets (extend as needed)
CANON = {
    "meals_per_day": "meals_per_day",
    "carbs_per_day": "carbs_per_day",
    "active_insulin_time_hours": "active_insulin_time_hours",
    "average_sg_mgdl": "average_sg_mgdl",
    "icr_1": "icr_1",
    "icr_2": "icr_2",
    "sg_sd_mgdl": "sg_sd_mgdl",
    "total_daily_dose_units": "total_daily_dose_units",
    # example duals for percent/non-percent:
    "hba1c": "hba1c",
    "hba1c_pct": "hba1c_pct",
}

# Aliases on canonical_key(raw) -> canonical name
ALIASES = {
    # meals
    canonical_key("meals"): "meals_per_day",
    canonical_key("meal"): "meals_per_day",
    canonical_key("meals/day"): "meals_per_day",
    canonical_key("meals per day"): "meals_per_day",
    canonical_key("number of meals"): "meals_per_day",

    # carbs
    canonical_key("carb"): "carbs_per_day",
    canonical_key("carbs"): "carbs_per_day",
    canonical_key("carb/day"): "carbs_per_day",
    canonical_key("carbs/day"): "carbs_per_day",
    canonical_key("carbohydrates per day"): "carbs_per_day",

    # specifics
    canonical_key("Active insulin time (hours)"): "active_insulin_time_hours",
    canonical_key("Active insulin time hours"): "active_insulin_time_hours",
    canonical_key("AIT (hours)"): "active_insulin_time_hours",

    canonical_key("Average SG mg/dL"): "average_sg_mgdl",
    canonical_key("Avg SG mg/dL"): "average_sg_mgdl",
    canonical_key("Average sensor glucose (mg/dL)"): "average_sg_mgdl",

    canonical_key("ICR-1"): "icr_1",
    canonical_key("ICR 1"): "icr_1",
    canonical_key("insulin carb ratio 1"): "icr_1",

    canonical_key("ICR-2"): "icr_2",
    canonical_key("ICR 2"): "icr_2",
    canonical_key("insulin carb ratio 2"): "icr_2",

    canonical_key("SG SD mg/dL"): "sg_sd_mgdl",
    canonical_key("sensor glucose sd (mg/dL)"): "sg_sd_mgdl",

    canonical_key("Total daily dose (Unit)"): "total_daily_dose_units",
    canonical_key("Total daily dose (Units)"): "total_daily_dose_units",
    canonical_key("TDD (units)"): "total_daily_dose_units",
    canonical_key("TDD"): "total_daily_dose_units",

    # HbA1c (non-percent vs percent — DO NOT MERGE)
    canonical_key("HbA1c"): "hba1c",
    canonical_key("HBA1C"): "hba1c",
    canonical_key("HbA1c (%)"): "hba1c_pct",
    canonical_key("hba1c%"): "hba1c_pct",
    canonical_key("hba1c%%"): "hba1c_pct",
}

def normalize_to_canonical(raw_name: str) -> str:
    """Map raw -> canonical (percent-safe)."""
    key = canonical_key(raw_name)
    return ALIASES.get(key, key)


# =========================
# ====== HELPERS ==========
# =========================
def _clean_df_strings(df: pd.DataFrame) -> pd.DataFrame:
    df2 = df.copy()
    for col in df2.columns:
        if df2[col].dtype == object:
            s = df2[col].astype(str).str.strip()
            s = s.replace({"": np.nan, "nan": np.nan, "NaN": np.nan, "NA": np.nan, "None": np.nan})
            df2[col] = s
    return df2

def percent_missing_by_column(df: pd.DataFrame, visit_name: str) -> pd.DataFrame:
    df2 = _clean_df_strings(df)
    miss = df2.isna().mean().mul(100).round(2)
    out = miss.reset_index()
    out.columns = ["Feature", f"% Missing {visit_name}"]
    return out

def missing_counts(df: pd.DataFrame, visit_name: str) -> pd.DataFrame:
    df2 = _clean_df_strings(df)
    total = len(df2)
    miss = df2.isna().sum()
    return pd.DataFrame({
        "Feature": miss.index,
        f"missing_{visit_name}": miss.values,
        f"total_{visit_name}": total
    })

def availability_for_features(features, visit_dfs):
    avail = []
    for feat in features:
        present_in = [vname for vname, vdf in visit_dfs.items() if feat in vdf.columns]
        avail.append(present_in)
    return pd.Series(avail, name="Availability (Visits)")

def _pick_feature_col(cols):
    for c in ["Feature", "Feature Name", "Variable", "Field", "Name"]:
        if c in cols:
            return c
    return None

def load_dictionary(dict_path: Path) -> pd.DataFrame | None:
    if not dict_path.exists():
        print(f"[INFO] Dictionary not found at {dict_path}. Skipping metadata merge.")
        return None

    try:
        if dict_path.suffix.lower() == ".csv":
            raw = pd.read_csv(dict_path)
        else:
            xls = pd.ExcelFile(dict_path)
            selected = None
            for s in xls.sheet_names:
                tmp = pd.read_excel(dict_path, sheet_name=s)
                fcol = _pick_feature_col(tmp.columns)
                if fcol and "Category" in tmp.columns:
                    selected = tmp.copy()
                    break
            if selected is None:
                selected = pd.read_excel(dict_path)
            raw = selected
    except Exception as e:
        print(f"[WARN] Failed to read dictionary: {e}. Skipping metadata merge.")
        return None

    fcol = _pick_feature_col(raw.columns)
    if fcol is None:
        print("[WARN] No recognizable feature-name column in dictionary. Skipping metadata merge.")
        return None

    df = raw.rename(columns={fcol: "Feature"})
    if "Definition" in df.columns and "Definition & Unit" not in df.columns:
        df = df.rename(columns={"Definition": "Definition & Unit"})
    keep_cols = [c for c in [
        "Feature", "Category", "Definition & Unit", "Unit/Type", "Feature Type", "Subtype"
    ] if c in df.columns]
    if "Feature" not in keep_cols or "Category" not in keep_cols:
        print("[WARN] Dictionary missing 'Feature' or 'Category' after normalization. Skipping metadata merge.")
        return None

    df = df[keep_cols].drop_duplicates()
    # percent-aware merge keys for the dictionary side
    df["Feature_base"]      = df["Feature"].map(clean_name_base)
    df["Feature_is_pct"]    = df["Feature"].map(has_percent)
    df["Feature_canonical"] = df["Feature"].map(normalize_to_canonical)
    return df

def weighted_overall_pct(merged_counts: pd.DataFrame) -> pd.Series:
    missing_cols = [c for c in merged_counts.columns if c.startswith("missing_")]
    total_cols   = [c for c in merged_counts.columns if c.startswith("total_")]
    missing_total = merged_counts[missing_cols].sum(axis=1)
    total_total   = merged_counts[total_cols].sum(axis=1)
    with np.errstate(divide="ignore", invalid="ignore"):
        pct = np.where(total_total > 0, (missing_total / total_total) * 100, np.nan)
    return np.round(pct, 2)

def qc_format_excel(writer, sheet_name, warn=WARN_THRES, alert=ALERT_THRES):
    """Add conditional formatting for % columns (>30 yellow, >50 red)."""
    try:
        from openpyxl.styles import PatternFill
        from openpyxl.formatting.rule import CellIsRule
        wb = writer.book
        ws = wb[sheet_name]
        header = [cell.value for cell in ws[1]]
        pct_cols = [i for i, h in enumerate(header, start=1) if isinstance(h, str) and "Missing" in h]
        yellow = PatternFill(start_color="FFF3CD", end_color="FFF3CD", fill_type="solid")  # warn
        red    = PatternFill(start_color="F8D7DA", end_color="F8D7DA", fill_type="solid")  # alert
        max_row = ws.max_row
        for col in pct_cols:
            col_letter = ws.cell(row=1, column=col).column_letter
            rng = f"{col_letter}2:{col_letter}{max_row}"
            ws.conditional_formatting.add(rng, CellIsRule(operator="greaterThan", formula=[str(warn)], fill=yellow))
            ws.conditional_formatting.add(rng, CellIsRule(operator="greaterThan", formula=[str(alert)], fill=red))
    except Exception as e:
        print(f"[INFO] QC formatting skipped on {sheet_name}: {e}")


# ------------- Main -------------
def main():
    # Ensure output dir exists
    OUT_XLSX.parent.mkdir(parents=True, exist_ok=True)

    # Load visit sheets
    xls = pd.ExcelFile(MASTER_PATH)
    visit_sheet_names = [s for s in xls.sheet_names if s not in SKIP_SHEETS]
    visits = {name: pd.read_excel(MASTER_PATH, sheet_name=name) for name in visit_sheet_names}

    # ---- Static: per-visit missingness ----
    miss_tables = [percent_missing_by_column(df, name) for name, df in visits.items()]
    from functools import reduce
    merged_miss = reduce(lambda L, R: pd.merge(L, R, on="Feature", how="outer"), miss_tables) if miss_tables else pd.DataFrame(columns=["Feature"])

    count_tables = [missing_counts(df, name) for name, df in visits.items()]
    merged_counts = reduce(lambda L, R: pd.merge(L, R, on="Feature", how="outer"), count_tables) if count_tables else pd.DataFrame(columns=["Feature"])
    for col in merged_counts.columns:
        if col.startswith("missing_") or col.startswith("total_"):
            merged_counts[col] = merged_counts[col].fillna(0)

    final_static = merged_miss.copy()
    if not merged_counts.empty:
        final_static["Overall % Missing"] = weighted_overall_pct(merged_counts)
    else:
        final_static["Overall % Missing"] = np.nan

    final_static["Availability (Visits)"] = availability_for_features(final_static["Feature"], visits).apply(lambda x: json.dumps(x))

    # ---- Merge dictionary (percent-aware) ----
    dict_core = load_dictionary(DICT_PATH)
    final_static["Feature_base"]      = final_static["Feature"].map(clean_name_base)
    final_static["Feature_is_pct"]    = final_static["Feature"].map(has_percent)
    final_static["Feature_canonical"] = final_static["Feature"].map(normalize_to_canonical)

    if dict_core is not None:
        # Pass 1: canonical join
        merged = final_static.merge(
            dict_core.add_suffix("_dict"),
            left_on="Feature_canonical",
            right_on="Feature_canonical_dict",
            how="left"
        )
        # Pass 2 (strict): fallback by (base, is_pct) for unmatched rows
        need_fb = merged["Category_dict"].isna() if "Category_dict" in merged.columns else pd.Series(False, index=merged.index)
        if need_fb.any():
            fb_left = final_static.loc[need_fb, ["Feature", "Feature_base", "Feature_is_pct"]].copy()
            fb_right = dict_core.add_suffix("_dict")[
                ["Feature_dict", "Feature_base_dict", "Feature_is_pct_dict",
                 "Category_dict", "Definition & Unit_dict", "Unit/Type_dict", "Feature Type_dict", "Subtype_dict"]
            ]
            fb_joined = fb_left.merge(
                fb_right,
                left_on=["Feature_base", "Feature_is_pct"],
                right_on=["Feature_base_dict", "Feature_is_pct_dict"],
                how="left"
            )
            idx = merged.index[need_fb]
            for col in fb_joined.columns:
                if col.endswith("_dict") and col in merged.columns:
                    merged.loc[idx, col] = fb_joined[col].values

        # Prefer dictionary metadata where available
        for c in ["Category", "Definition & Unit", "Unit/Type", "Feature Type", "Subtype"]:
            lc, rc = c, f"{c}_dict"
            if lc not in merged.columns:
                merged[lc] = np.nan
            if rc in merged.columns:
                merged[lc] = merged[lc].combine_first(merged[rc])

        merged["Matched Dictionary Feature"] = merged.get("Feature_dict", np.nan)
        # Cleanup helper cols
        drop_helpers = [c for c in merged.columns if c.endswith("_dict")] + \
                       ["Feature_base", "Feature_base_dict", "Feature_is_pct", "Feature_is_pct_dict",
                        "Feature_canonical", "Feature_canonical_dict"]
        final_static = merged.drop(columns=[c for c in drop_helpers if c in merged.columns])
    else:
        for c in ["Category", "Definition & Unit", "Unit/Type", "Feature Type", "Subtype", "Matched Dictionary Feature"]:
            if c not in final_static.columns:
                final_static[c] = np.nan

    # Order columns
    visit_cols = [c for c in final_static.columns if c.startswith("% Missing ")]
    final_static = final_static[
        ["Category", "Feature", "Matched Dictionary Feature", "Definition & Unit",
         "Unit/Type", "Feature Type", "Subtype"]
        + visit_cols
        + ["Overall % Missing", "Availability (Visits)"]
    ]

    # ---- Ramadan/Shawwal rollups (static) ----
    counts_map = {name: missing_counts(df, name).set_index("Feature") for name, df in visits.items()}

    def pooled_group_pct(features, group_visits):
        if len(features) == 0:
            return np.array([])
        miss_total = pd.Series(0, index=features, dtype=float)
        tot_total  = pd.Series(0, index=features, dtype=float)
        for v in group_visits:
            if v not in counts_map:
                continue
            mc = counts_map[v]
            miss_total = miss_total.add(mc.get(f"missing_{v}", pd.Series(0, index=features)).reindex(features).fillna(0), fill_value=0)
            tot_total  = tot_total.add(mc.get(f"total_{v}", pd.Series(0, index=features)).reindex(features).fillna(0),   fill_value=0)
        with np.errstate(divide="ignore", invalid="ignore"):
            pct = np.where(tot_total > 0, (miss_total / tot_total) * 100, np.nan)
        return np.round(pct, 2)

    features_list = final_static["Feature"]
    final_static["% Missing Ramadan (rollup)"] = pooled_group_pct(features_list, RAMADAN_VISITS)
    final_static["% Missing Shawwal (rollup)"] = pooled_group_pct(features_list, SHAWWAL_VISITS)

    # ---- Patient-level missingness per visit (row-wise) ----
    plm_list = []
    for vname, vdf in visits.items():
        if PATIENT_ID_COL not in vdf.columns:
            continue
        df2 = _clean_df_strings(vdf)
        cols_eval = [c for c in df2.columns if c != PATIENT_ID_COL]
        if not cols_eval:
            continue
        row_pct = df2[cols_eval].isna().mean(axis=1).mul(100).round(2)
        tmp = pd.DataFrame({
            "Visit": vname,
            PATIENT_ID_COL: df2[PATIENT_ID_COL],
            "Row % Missing (all fields)": row_pct,
            "Missing Cells": df2[cols_eval].isna().sum(axis=1),
            "Total Cells": len(cols_eval)
        })
        plm_list.append(tmp)
    patient_level_missing = pd.concat(plm_list, ignore_index=True) if plm_list else pd.DataFrame()

    # ---- Dynamic (intraday) Ramadan/Shawwal ----
    final_dynamic = pd.DataFrame()
    if INCLUDE_INTRADAY and INTRADAY_CSV_PATH.exists():
        try:
            intraday = pd.read_csv(INTRADAY_CSV_PATH)
            if "date" in intraday.columns:
                intraday["date"] = pd.to_datetime(intraday["date"], errors="coerce")
                ram = intraday[(intraday["date"] >= RAMADAN_START) & (intraday["date"] <= RAMADAN_END)]
                shw = intraday[(intraday["date"] >= SHAWWAL_START) & (intraday["date"] <= SHAWWAL_END)]

                dyn_cols = [c for c in intraday.columns if c not in INTRADAY_EXCLUDE_COLS]

                rows = []
                for feat in dyn_cols:
                    r_miss = ram[feat].isna().mean() * 100 if len(ram) else np.nan
                    s_miss = shw[feat].isna().mean() * 100 if len(shw) else np.nan
                    both = pd.concat([ram[feat], shw[feat]]) if len(ram) or len(shw) else pd.Series(dtype=float)
                    overall = both.isna().mean() * 100 if len(both) else np.nan
                    rows.append({
                        "Feature": feat,
                        "% Missing Ramadan": round(r_miss, 2) if pd.notna(r_miss) else np.nan,
                        "% Missing Shawwal": round(s_miss, 2) if pd.notna(s_miss) else np.nan,
                        "Overall % Missing": round(overall, 2) if pd.notna(overall) else np.nan,
                        "Availability (Visits)": json.dumps(["Ramadan (intraday)", "Shawwal (intraday)"])
                    })
                final_dynamic = pd.DataFrame(rows)

                # Dictionary enrichment (percent-aware)
                dict_core_dyn = dict_core  # reuse if available
                if dict_core_dyn is not None and not final_dynamic.empty:
                    final_dynamic["Feature_base"]      = final_dynamic["Feature"].map(clean_name_base)
                    final_dynamic["Feature_is_pct"]    = final_dynamic["Feature"].map(has_percent)
                    final_dynamic["Feature_canonical"] = final_dynamic["Feature"].map(normalize_to_canonical)

                    merged_dyn = final_dynamic.merge(
                        dict_core_dyn.add_suffix("_dict"),
                        left_on="Feature_canonical",
                        right_on="Feature_canonical_dict",
                        how="left"
                    )
                    # strict fallback on (base, is_pct)
                    need_fb = merged_dyn["Category_dict"].isna() if "Category_dict" in merged_dyn.columns else pd.Series(False, index=merged_dyn.index)
                    if need_fb.any():
                        fb_left = final_dynamic.loc[need_fb, ["Feature", "Feature_base", "Feature_is_pct"]].copy()
                        fb_right = dict_core_dyn.add_suffix("_dict")[
                            ["Feature_dict", "Feature_base_dict", "Feature_is_pct_dict",
                             "Category_dict", "Definition & Unit_dict", "Unit/Type_dict", "Feature Type_dict", "Subtype_dict"]
                        ]
                        fb_joined = fb_left.merge(
                            fb_right,
                            left_on=["Feature_base", "Feature_is_pct"],
                            right_on=["Feature_base_dict", "Feature_is_pct_dict"],
                            how="left"
                        )
                        idx = merged_dyn.index[need_fb]
                        for col in fb_joined.columns:
                            if col.endswith("_dict") and col in merged_dyn.columns:
                                merged_dyn.loc[idx, col] = fb_joined[col].values

                    for c in ["Category", "Definition & Unit", "Unit/Type", "Feature Type", "Subtype"]:
                        lc, rc = c, f"{c}_dict"
                        if lc not in merged_dyn.columns:
                            merged_dyn[lc] = np.nan
                        if rc in merged_dyn.columns:
                            merged_dyn[lc] = merged_dyn[lc].combine_first(merged_dyn[rc])

                    merged_dyn["Matched Dictionary Feature"] = merged_dyn.get("Feature_dict", np.nan)
                    drop_helpers = [c for c in merged_dyn.columns if c.endswith("_dict")] + \
                                   ["Feature_base", "Feature_base_dict", "Feature_is_pct", "Feature_is_pct_dict",
                                    "Feature_canonical", "Feature_canonical_dict"]
                    final_dynamic = merged_dyn.drop(columns=[c for c in drop_helpers if c in merged_dyn.columns])
                else:
                    for c in ["Category", "Definition & Unit", "Unit/Type", "Feature Type", "Subtype", "Matched Dictionary Feature"]:
                        if c not in final_dynamic.columns:
                            final_dynamic[c] = np.nan
            else:
                print("[INFO] intraday has no 'date' column; dynamic missingness skipped.")
        except Exception as e:
            print(f"[WARN] Intraday integration skipped: {e}")

    # ---- Save everything to Excel with QC formatting ----
    with pd.ExcelWriter(OUT_XLSX, engine="openpyxl") as writer:
        # Static (always create at least one sheet to avoid openpyxl 'no visible sheet' error)
        final_static.to_excel(writer, index=False, sheet_name="Missingness_Static")
        qc_format_excel(writer, "Missingness_Static")

        # Dynamic (Ramadan/Shawwal)
        if not final_dynamic.empty:
            dyn_cols_order = ["Category", "Feature", "Matched Dictionary Feature", "Definition & Unit",
                              "Unit/Type", "Feature Type", "Subtype",
                              "% Missing Ramadan", "% Missing Shawwal", "Overall % Missing", "Availability (Visits)"]
            dyn_cols_order = [c for c in dyn_cols_order if c in final_dynamic.columns] + \
                             [c for c in final_dynamic.columns if c not in dyn_cols_order]
            final_dynamic[dyn_cols_order].to_excel(writer, index=False, sheet_name="Missingness_Dynamic")
            qc_format_excel(writer, "Missingness_Dynamic")

        # Dynamic-by-Visit (optional stage 2 CSV)
        if DYN_VISIT_OUT.exists():
            try:
                dyn_visit_df = pd.read_csv(DYN_VISIT_OUT)
                dyn_visit_df.to_excel(writer, index=False, sheet_name="Missingness_Dynamic_By_Visit")
                qc_format_excel(writer, "Missingness_Dynamic_By_Visit")
                print("✅ Added sheet: Missingness_Dynamic_By_Visit")
            except Exception as e:
                print(f"[INFO] Skipped Missingness_Dynamic_By_Visit: {e}")

        # Patient-level
        if not patient_level_missing.empty:
            patient_level_missing.to_excel(writer, index=False, sheet_name="Patient_Level_Missingness")

        # Unmapped audits
        if "Category" in final_static.columns:
            unmapped_static = (
                final_static[final_static["Category"].isna()][["Feature"]]
                .drop_duplicates().sort_values("Feature")
            )
            unmapped_static.to_excel(writer, index=False, sheet_name="Unmapped_Static")

        if not final_dynamic.empty and "Category" in final_dynamic.columns:
            unmapped_dyn = (
                final_dynamic[final_dynamic["Category"].isna()][["Feature"]]
                .drop_duplicates().sort_values("Feature")
            )
            unmapped_dyn.to_excel(writer, index=False, sheet_name="Unmapped_Dynamic")

        # Name mapping audits
        if "Matched Dictionary Feature" in final_static.columns:
            mapping_static = (
                final_static[["Feature", "Matched Dictionary Feature"]]
                .drop_duplicates().sort_values(["Feature", "Matched Dictionary Feature"])
            )
            mapping_static.to_excel(writer, index=False, sheet_name="Name_Mapping_Static")

        if not final_dynamic.empty and "Matched Dictionary Feature" in final_dynamic.columns:
            mapping_dyn = (
                final_dynamic[["Feature", "Matched Dictionary Feature"]]
                .drop_duplicates().sort_values(["Feature", "Matched Dictionary Feature"])
            )
            mapping_dyn.to_excel(writer, index=False, sheet_name="Name_Mapping_Dynamic")

    # CSVs
    final_static.to_csv(OUT_CSV_STATIC, index=False)
    if not final_dynamic.empty:
        final_dynamic.to_csv(OUT_CSV_DYNAMIC, index=False)
    if not patient_level_missing.empty:
        patient_level_missing.to_csv(OUT_CSV_PATIENT, index=False)

    print(f"Saved Excel report → {OUT_XLSX}")
    print(f"Saved static CSV   → {OUT_CSV_STATIC}")
    if not final_dynamic.empty:
        print(f"Saved dynamic CSV  → {OUT_CSV_DYNAMIC}")
    if not patient_level_missing.empty:
        print(f"Saved patient CSV  → {OUT_CSV_PATIENT}")


if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd

# Load selected columns from the full sheet
df = pd.read_excel(
    "/kaggle/working/intraday_with_visits.xlsx",  # ← change to your real input folder",  # ← change to your real input folder
    sheet_name="Intraday_All",
    usecols=["patientID", "start", "cgm"]
)

# Convert start to datetime and filter for Ramadan
df["start"] = pd.to_datetime(df["start"], errors="coerce")
df = df[(df["start"] >= "2023-03-22") & (df["start"] <= "2023-04-19")]

# Drop missing rows
df = df.dropna(subset=["patientID", "cgm"])

# Sample 60,000 rows
df_sample = df.sample(n=min(60000, len(df)), random_state=42)

# Save to Excel for later upload
df_sample.to_excel("/kaggle/working/intraday_ramadan_subset.xlsx", index=False)


# 4. static baseline covariates (patient-level features)

This code analyzes static baseline covariates (patient-level features) from the Excel file final_master_sheet_clean.xlsx.
It performs the following steps:

Loads and merges all sheets from the master Excel into one dataset.

Cleans column names and automatically detects key baseline features (age, gender, BMI, HbA1C, cholesterol, LDL, HDL, triglycerides, SBP, DBP, eGFR, creatinine, insulin/kg, SmartGuard %).

1. Computes descriptive statistics (mean, median, SD, min, max) for all numeric variables.

2. Calculates a correlation matrix to identify relationships among covariates.

3. Visualizes relationships with a pairplot (scatter and KDE) and a heatmap of correlations.

4. Exports results as two CSV files:

5. Static_Baseline_Descriptive_Stats.csv

6. Static_Baseline_Correlation.csv

In summary, it provides a full exploratory and correlation analysis of baseline clinical and metabolic variables across all patient visits.


This code performs a **cleaned, patient-level analysis** of static baseline covariates from the Excel file `final_master_sheet_clean.xlsx`.

### 🔍 Step-by-step summary

## 1. **Load all visits**

   * Reads every sheet from the Excel file.
   * Combines them into one DataFrame, adding a `visit` column to track each visit.

## 2. **Normalize column names**

   * Converts all headers to lowercase and removes extra spaces for consistency.

## 3. **Identify the patient identifier**

   * Automatically detects the column containing patient IDs (e.g., “patientid” or “patient code”).

## 4. **Select only static baseline covariates**

   * Keeps columns related to demographics and lab results such as:

     * Age, gender/sex, BMI
     * HbA1C
     * Lipids (cholesterol, LDL, HDL, triglycerides)
     * Blood pressure (SBP, DBP)
     * Kidney markers (eGFR, creatinine)
     * Insulin/kg, SmartGuard %

## 5. **Aggregate to one row per patient**

   * For each patient:

     * Numeric features → median value across visits
     * Categorical features (like gender) → most frequent value (mode)

##  6. **Descriptive statistics**

   * Prints summary metrics (mean, median, standard deviation, etc.) for all baseline variables.

## 7. **Correlation heatmap**

   * Computes correlations among numeric features (e.g., LDL vs. cholesterol, eGFR vs. creatinine).
   * Displays them visually using a color-coded heatmap.

---

### ✅ **In short**

The script consolidates all visit data into a **single baseline record per patient**, summarizes their demographic and clinical characteristics, and visualizes **inter-variable correlations** to understand relationships among core metabolic and renal covariates.


In [ ]:
# Study the Static Baseline Covariates (patient-level) from the master Excel file.
# We'll compute descriptive statistics, correlations, and visualize relationships.

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

# Load the uploaded Excel
file_path = "/kaggle/input/static-variables/final_master_sheet_clean.xlsx"

# Read all sheets
xls = pd.ExcelFile(file_path)
sheets = xls.sheet_names
print("Sheets found:", sheets)

# Aggregate all visits into one DataFrame for static features
dfs = []
for s in sheets:
    df = pd.read_excel(file_path, sheet_name=s)
    df["visit_sheet"] = s
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

# Normalize column names for consistency
df_all.columns = df_all.columns.str.strip().str.lower()

# Identify key features (match by lowercase name fragments)
features = {
    "age": [c for c in df_all.columns if "age" in c],
    "gender": [c for c in df_all.columns if "gender" in c or "sex" in c],
    "bmi": [c for c in df_all.columns if "bmi" in c],
    "hba1c": [c for c in df_all.columns if "hba1" in c],
    "cholesterol": [c for c in df_all.columns if "cholesterol" in c],
    "ldl": [c for c in df_all.columns if "ldl" in c],
    "hdl": [c for c in df_all.columns if "hdl" in c],
    "triglyceride": [c for c in df_all.columns if "trig" in c],
    "sbp": [c for c in df_all.columns if "sbp" in c or "systolic" in c],
    "dbp": [c for c in df_all.columns if "dbp" in c or "diastolic" in c],
    "egfr": [c for c in df_all.columns if "egfr" in c],
    "creatinine": [c for c in df_all.columns if "creat" in c],
    "insulin_kg": [c for c in df_all.columns if "insulin" in c and "kg" in c],
    "smartguard": [c for c in df_all.columns if "smartguard" in c],
}

# Flatten to a feature list and extract relevant subset
selected_cols = [col[0] for col in features.values() if len(col) > 0]
static_df = df_all[selected_cols].copy()

# Descriptive statistics
desc = static_df.describe(include="all").T

# Correlation matrix for numeric columns
numeric_df = static_df.select_dtypes(include=[np.number])
corr = numeric_df.corr()

# --- Visualization ---
sns.set(style="whitegrid", context="notebook")

# 1. Pairplot for main features
pair_cols = ["bmi", "hba1c", "cholesterol", "ldl", "hdl", "triglyceride", "egfr", "creatinine"]
pair_cols = [c for c in pair_cols if c in numeric_df.columns]
sns.pairplot(static_df[pair_cols], diag_kind="kde")
plt.suptitle("Static Baseline Covariates Relationships", y=1.02)
plt.tight_layout()
plt.show()

# 2. Correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", cbar_kws={'label': 'Correlation'})
plt.title("Correlation Heatmap – Static Baseline Covariates")
plt.tight_layout()
plt.show()

# Save outputs
out_dir = Path("/kaggle/working/static_baseline_analysis")
out_dir.mkdir(parents=True, exist_ok=True)
desc_path = out_dir / "Static_Baseline_Descriptive_Stats.csv"
corr_path = out_dir / "Static_Baseline_Correlation.csv"

desc.to_csv(desc_path)
corr.to_csv(corr_path)

desc.head(), corr.head(), {"descriptive_csv": str(desc_path), "correlation_csv": str(corr_path)}


In [ ]:
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt

# --- Load the Excel ---
xls = pd.ExcelFile("/kaggle/input/static-variables/final_master_sheet_clean.xlsx")

# Combine all visits (so we can take per-patient medians later)
df_all = pd.concat([pd.read_excel(xls, s).assign(visit=s) for s in xls.sheet_names], ignore_index=True)

# Normalize column names
df_all.columns = df_all.columns.str.strip().str.lower()

# Identify the patient ID column
pid_col = [c for c in df_all.columns if "patient" in c or "code" in c][0]

# --- Select only static baseline covariates ---
keep_patterns = [
    "age", "gender", "sex", "bmi", "hba1", "cholesterol", "ldl", "hdl", "trig",
    "sbp", "dbp", "egfr", "creat", "insulin", "smartguard"
]
cols = [c for c in df_all.columns if any(p in c for p in keep_patterns)]
static_df = df_all[[pid_col] + cols].copy()

# --- Aggregate to one row per patient (median for numeric, mode for gender) ---
static_df = static_df.groupby(pid_col).agg(lambda x: pd.to_numeric(x, errors="coerce").median(skipna=True)
                                           if np.issubdtype(x.dtype, np.number) or str(x.dtype).startswith("float")
                                           else x.mode().iloc[0] if not x.mode().empty else x.iloc[0]).reset_index()

print("Number of unique patients:", static_df.shape[0])

# --- Descriptive statistics ---
desc = static_df.describe(include='all').T
print("\nDescriptive summary:\n", desc)

# --- Correlation heatmap for numeric features ---
num_df = static_df.select_dtypes(include=[np.number])

plt.figure(figsize=(15, 10))
sns.heatmap(num_df.corr(), annot=True, fmt=".2f", cmap="coolwarm", cbar_kws={'label': 'Correlation'})
plt.title("Static Baseline Covariates – Unique Patients (n ≈ 33–35)")
plt.tight_layout()
plt.show()


# 5. Dynamic Features during Ramadan Visits (Visits 2–5) f

This script extracts and summarizes the **dynamic features during Ramadan visits (Visits 2–5)** from your `final_master_sheet_clean.xlsx`.

Here’s what it does, step by step:

---

## 🧭 **1. Load and Filter Ramadan Visits**

* Reads every sheet in the Excel file.
* Keeps only those whose sheet names include **2, 3, 4, 5** (Ramadan visits).
* Combines them into one DataFrame with a `visit` column.

```python
Ramadan sheets: ['Visit 2', 'Visit 3', 'Visit 4', 'Visit 5']
```

---

## 🧹 **2. Normalize Column Names**

All column names are converted to lowercase and stripped of spaces so matching is consistent.

---

## ⚙️ **3. Define Feature Groups**

Defines three clinical domains (based on your 1.3 variable specification):

| Domain               | Example Features Detected                                          |
| -------------------- | ------------------------------------------------------------------ |
| **Glycemia**         | `TIR 70–180`, `Total T<70`, `SG SD`, `CV %`, `GRI`                 |
| **Insulin / Device** | `Total Daily Dose`, `Bolus %`, `Auto Basal %`, `Auto Correction %` |
| **Lifestyle**        | `Meals/Day`, `Carb/Day`, `Fasting %`, `Fasting Days`               |

The helper function `find_columns()` scans column names and collects all matches for these patterns.


### ✅ **In short**

> This code isolates and prepares **Ramadan-phase dynamic clinical and behavioral variables** (glycemia, insulin/device metrics, and lifestyle factors) for downstream modeling—ensuring clean numeric formatting, per-visit structure, and quick statistical and correlation summaries.

Here’s a **comprehensive, unified summary** of all the major steps — combining the three scripts into a single clear workflow overview 👇

---

## 🌙 **Ramadan Dynamic Feature Extraction, Cleaning, and Modeling Pipeline**

This complete workflow extracts, prepares, and models **dynamic clinical and behavioral features** from Ramadan visits (Visits 2–5) in the study dataset `final_master_sheet_clean.xlsx`.
It transforms raw visit-level data into a **clean, numeric, and modeling-ready dataset** and uses machine learning to predict **hypoglycemia burden (Total T<70%)** during Ramadan.

---

### 🧭 **1️⃣ Extract and Combine Ramadan Visits**

* Reads all sheets in the master Excel file.
* Keeps only **Visits 2, 3, 4, and 5** (Ramadan phase).
* Merges them into one unified DataFrame and adds a `visit` column.
  👉 *Purpose:* Consolidate all Ramadan data across patients into one dataset.

---

### 🧹 **2️⃣ Clean and Normalize Columns**

* Converts all column names to lowercase and removes spaces or symbols.
* Ensures consistent column matching across visits regardless of formatting differences.
  👉 *Purpose:* Standardize variable names for accurate identification.

---

### ⚙️ **3️⃣ Define and Identify Dynamic Feature Groups**

Automatically detects features belonging to three major **clinical domains**:

| Domain             | Example Features                                                   | Description                                             |
| :----------------- | :----------------------------------------------------------------- | :------------------------------------------------------ |
| **Glycemia**       | `TIR 70–180`, `Total T<70`, `SG SD`, `CV %`, `GRI`                 | Continuous glucose and variability metrics.             |
| **Insulin/Device** | `Total Daily Dose`, `Bolus %`, `Auto Basal %`, `Auto Correction %` | Insulin delivery and pump metrics.                      |
| **Lifestyle**      | `Meals/Day`, `Carb/Day`, `Fasting %`, `Fasting Days`               | Meal frequency, carbohydrate intake, fasting adherence. |

A helper function scans all columns and collects matches for each group automatically.
👉 *Purpose:* Automatically categorize clinical and behavioral variables for analysis.

---

### 🔍 **4️⃣ Select, Clean, and Save Dynamic Features**

* Automatically identifies the **patient ID** column.
* Keeps only patient ID, visit, and matched dynamic variables.
* Converts all numeric-like columns to numeric dtype.
* Saves the clean Ramadan dynamic dataset to:

  ```
  /kaggle/working/Dynamic_Features_Ramadan.csv
  ```

👉 *Purpose:* Produce a structured, numeric dataset per patient/visit for statistical and modeling tasks.

---

### 📊 **5️⃣ Descriptive Statistics and Correlations**

* Generates descriptive summaries (count, mean, SD, min, quartiles, max).
* Computes a correlation matrix between all dynamic variables to check inter-feature relationships.
  👉 *Purpose:* Understand data distribution and detect collinearity between variables.

---

### 🧩 **6️⃣ Feature Selection for Modeling**

* Groups features into:

  * **Keep** → Insulin/Device + Lifestyle
  * **Drop** → Glycemia (to prevent information leakage into glycemic outcomes).
* Visualizes these decisions with a color-coded scatterplot (green = keep, red = drop).
* Exports a refined modeling dataset:

  ```
  /kaggle/working/Dynamic_Features_Ramadan_Clean.csv
  ```

👉 *Purpose:* Retain only independent behavioral and insulin-related predictors for modeling.

---

### 🤖 **7️⃣ Machine Learning Modeling (Hypoglycemia Prediction)**

Two ensemble regressors are trained to predict **Total T<70 %** (time below 70 mg/dL):

| Model                       | Key Parameters                                 | Evaluation Metrics                                         |
| :-------------------------- | :--------------------------------------------- | :--------------------------------------------------------- |
| **Random Forest Regressor** | 300 trees, random state = 42                   | R² and RMSE on test data; feature importance = split gain. |
| **XGBoost Regressor**       | 300 trees, learning rate = 0.05, max depth = 3 | R² and RMSE; feature importance = gradient gain.           |

* Data split: 75 % train / 25 % test.
* Only numeric variables are used.
* Both models report:

  * **R²** → fraction of explained variance.
  * **RMSE** → prediction error magnitude.
* Feature importances are ranked and visualized side-by-side for RF vs XGB.
  👉 *Purpose:* Evaluate how well insulin and lifestyle patterns explain hypoglycemia risk and identify top predictive features.

---

### 📈 **8️⃣ Outputs**

| Output                                               | Description                                                                     |
| :--------------------------------------------------- | :------------------------------------------------------------------------------ |
| `/kaggle/working/Dynamic_Features_Ramadan.csv`       | Full Ramadan dynamic dataset.                                                   |
| `/kaggle/working/Dynamic_Features_Ramadan_Clean.csv` | Modeling-ready subset (insulin + lifestyle features).                           |
| Model Metrics                                        | R² and RMSE for Random Forest and XGBoost.                                      |
| Feature Importance Plots                             | Top predictors of hypoglycemia (e.g., Bolus %, Auto Basal %, Total Daily Dose). |

---

## ✅ **Overall Summary**

> The complete pipeline extracts Ramadan-phase data from multi-visit Excel sheets, standardizes and classifies dynamic variables into glycemia, insulin/device, and lifestyle domains, selects relevant predictors, and trains Random Forest and XGBoost models to estimate **hypoglycemia exposure (Total T<70 %)**.
> It produces clean, interpretable data, quantitative summaries, and feature-importance insights for subsequent clinical interpretation and modeling.


In [ ]:
import pandas as pd, numpy as np

# --- Load workbook with all visits ---
xls = pd.ExcelFile("/kaggle/input/static-variables/final_master_sheet_clean.xlsx")

# --- Restrict to Ramadan visits (2–5) ---
ramadan_sheets = [s for s in xls.sheet_names if any(str(i) in s for i in [2,3,4,5])]
print("Ramadan sheets:", ramadan_sheets)

# Combine visits into one DataFrame
ramadan_df = pd.concat([pd.read_excel(xls, s).assign(visit=s) for s in ramadan_sheets], ignore_index=True)

# Normalize column names
ramadan_df.columns = ramadan_df.columns.str.strip().str.lower()

# --- Define feature groups based on your 1.3 spec ---
glycemia_vars = ["tir 70 - 180", "tir70-180", "tir_70_180", "total t<70", "t<70",
                 "sg sd", "sgsd", "coefficient of variation", "coefficientofvariation", "gri"]

insulin_device_vars = ["total daily dose", "tdd", "bolus %", "bolus%", 
                       "auto basal %", "autobasal%", "auto correction %", "autocorrection%"]

lifestyle_vars = ["meals/day", "meals", "carb/day", "carb", "fasting %", "fasting days"]

# Helper: find first matching column per pattern
def find_columns(df, patterns):
    found = []
    for c in df.columns:
        for p in patterns:
            if p.replace(" ", "") in c.replace(" ", ""):
                found.append(c)
                break
    return list(dict.fromkeys(found))  # unique, preserve order

gly_cols = find_columns(ramadan_df, glycemia_vars)
insulin_cols = find_columns(ramadan_df, insulin_device_vars)
life_cols = find_columns(ramadan_df, lifestyle_vars)

dynamic_cols = gly_cols + insulin_cols + life_cols
print("Selected dynamic feature columns:", dynamic_cols)

# --- Build final modeling DataFrame ---
pid_col = [c for c in ramadan_df.columns if "patient" in c or "code" in c][0]
dyn_df = ramadan_df[[pid_col, "visit"] + dynamic_cols].copy()

# Convert all numeric features to numeric dtype
for c in dynamic_cols:
    dyn_df[c] = pd.to_numeric(dyn_df[c], errors="coerce")

# --- Descriptive summary & correlation check ---
desc = dyn_df[dynamic_cols].describe().T
corr = dyn_df[dynamic_cols].corr()

print("\nDynamic feature descriptive summary (median ± IQR):\n", desc)
print("\nPairwise correlations:\n", corr.round(2))

# --- Optional: save for modeling ---
out_path = "/kaggle/working/Dynamic_Features_Ramadan.csv"
dyn_df.to_csv(out_path, index=False)
print(f"\nSaved Ramadan dynamic feature dataset → {out_path}")


In [ ]:
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Load your Ramadan dynamic dataset
df = pd.read_csv("/kaggle/working/Dynamic_Features_Ramadan.csv")

# --- Define Glycemia subset ---
glycemia = ["tir 70 - 180", "total t<70", "sg sd", "coefficient of variation", "gri"]
glycemia = [g for g in glycemia if g in df.columns]

# Drop rows with all NaN in glycemia block
gly_df = df[glycemia].dropna(how='all')

# --- 1️⃣ Correlation Heatmap ---
plt.figure(figsize=(7,5))
sns.heatmap(gly_df.corr(), annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1, cbar_kws={'label':'Correlation'})
plt.title("Correlation Heatmap – Glycemia Features (Ramadan)")
plt.tight_layout()
plt.show()

# --- 2️⃣ VIF (Variance Inflation Factor) ---
# Standardize numeric matrix
X = StandardScaler().fit_transform(gly_df.dropna())
vif_df = pd.DataFrame()
vif_df["Feature"] = glycemia
vif_df["VIF"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
print("\nVariance Inflation Factors (VIF):\n", vif_df)

# --- 3️⃣ Diagram / annotation of keep vs drop ---
# Rule: drop features with |r| > 0.9 OR VIF > 10
corr_mat = gly_df.corr().abs()
upper = corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]

# Highlight decisions
keep_features = [f for f in glycemia if f not in to_drop]
drop_features = to_drop

print("\n🧮 Features to KEEP:", keep_features)
print("🚫 Features to DROP (highly redundant):", drop_features)

# --- 4️⃣ Diagrammatic summary ---
fig, ax = plt.subplots(figsize=(6,4))
sns.scatterplot(x=["keep"]*len(keep_features)+["drop"]*len(drop_features),
                y=keep_features+drop_features,
                hue=["Keep"]*len(keep_features)+["Drop"]*len(drop_features),
                palette={"Keep":"green","Drop":"red"}, s=200)
for i, f in enumerate(keep_features+drop_features):
    ax.text(-0.1 if i < len(keep_features) else 0.9, i, f, va="center", fontsize=10)
ax.set_title("Feature Selection Decision – Glycemia Features (Ramadan)")
ax.set_xlabel("Decision Category")
ax.set_ylabel("")
ax.legend(title="Status", loc="best")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd, seaborn as sns, matplotlib.pyplot as plt

# Load the Ramadan dataset
df = pd.read_csv("/kaggle/working/Dynamic_Features_Ramadan.csv")

# Define feature categories
glycemia_features = ["tir 70 - 180", "total t<70", "sg sd", "coefficient of variation", "gri"]
insulin_device_features = ["total daily dose (u)", "bolus %", "auto basal %", "auto correction %"]
lifestyle_features = ["meals", "carb", "fasting % (out of 29 days)"]

# Keep only the features that exist in your dataset
keep_features = [f for f in insulin_device_features + lifestyle_features if f in df.columns]
drop_features = [f for f in glycemia_features if f in df.columns]

# Assign group labels only for features that exist
group_map = {}
for f in keep_features:
    group_map[f] = "Insulin/Device" if f in insulin_device_features else "Lifestyle/Meals"
for f in drop_features:
    group_map[f] = "Glycemia"

# Build the decision DataFrame
selection_df = pd.DataFrame({
    "Feature": keep_features + drop_features,
    "Group": [group_map[f] for f in keep_features + drop_features],
    "Decision": ["Keep"] * len(keep_features) + ["Drop"] * len(drop_features)
})

# --- Diagram ---
plt.figure(figsize=(7, 5))
sns.scatterplot(
    data=selection_df,
    x="Decision", y="Feature",
    hue="Decision", style="Group",
    palette={"Keep": "green", "Drop": "red"},
    s=200
)
plt.title("Feature Selection Decision – Ramadan Dynamic Features")
plt.xlabel("Decision Category")
plt.ylabel("")
plt.legend(title="Decision / Group", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

# --- Export clean dataset ---
pid_cols = [c for c in df.columns if "patient" in c or "code" in c] + ["visit"]
df_clean = df[pid_cols + keep_features]
out_path = "/kaggle/working/Dynamic_Features_Ramadan_Clean.csv"
df_clean.to_csv(out_path, index=False)

print("✅ Kept features for modeling:", keep_features)
print("🚫 Dropped glycemia features:", drop_features)
print(f"Cleaned feature dataset saved → {out_path}")


In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error

# --- Load your clean Ramadan dataset ---
df = pd.read_csv("/kaggle/working/Dynamic_Features_Ramadan_Clean.csv")

# Load original dataset to bring back target (total T<70)
orig = pd.read_csv("/kaggle/working/Dynamic_Features_Ramadan.csv")

# Merge so we can predict total T<70 %
target_col = "total t<70"
df = pd.merge(df, orig[[target_col]], left_index=True, right_index=True, how="left")

# Drop rows with any missing values (NaN)
df = df.dropna()

# --- Define features (X) and target (y) ---
X = df.drop(columns=[target_col])
y = df[target_col]

# Keep only numeric columns for modeling
X = X.select_dtypes(include=[np.number])

# Split data (train/test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# --- 1️⃣ Random Forest Regressor ---
rf = RandomForestRegressor(n_estimators=300, random_state=42)
rf.fit(X_train, y_train)

# Predict & evaluate
y_pred_rf = rf.predict(X_test)
print(f"Random Forest R²: {r2_score(y_test, y_pred_rf):.3f} | RMSE: {mean_squared_error(y_test, y_pred_rf, squared=False):.3f}")

# Feature importance
fi_rf = pd.DataFrame({"Feature": X.columns, "Importance": rf.feature_importances_}).sort_values("Importance", ascending=False)
print("\nRandom Forest Feature Importance:\n", fi_rf)

# --- 2️⃣ XGBoost Regressor ---
xgb = XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42)
xgb.fit(X_train, y_train)

# Predict & evaluate
y_pred_xgb = xgb.predict(X_test)
print(f"\nXGBoost R²: {r2_score(y_test, y_pred_xgb):.3f} | RMSE: {mean_squared_error(y_test, y_pred_xgb, squared=False):.3f}")

# Feature importance
fi_xgb = pd.DataFrame({"Feature": X.columns, "Importance": xgb.feature_importances_}).sort_values("Importance", ascending=False)
print("\nXGBoost Feature Importance:\n", fi_xgb)

# --- 📊 Plot both importance sets ---
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
sns.barplot(data=fi_rf, x="Importance", y="Feature", ax=axes[0], palette="Blues_d")
axes[0].set_title("Random Forest – Feature Importance")

sns.barplot(data=fi_xgb, x="Importance", y="Feature", ax=axes[1], palette="Greens_d")
axes[1].set_title("XGBoost – Feature Importance")

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd, numpy as np, re, unicodedata
from pathlib import Path

# ========= CONFIG =========
IN_XLSX  = Path("/kaggle/working/final_master_sheet_clean_with_visits.xlsx")
OUT_XLSX = Path("/kaggle/working/outcomes_split_static_vs_visit.xlsx")
OUT_STATIC_CSV      = Path("/kaggle/working/outcome_static.csv")
OUT_VISIT_LONG_CSV  = Path("/kaggle/working/outcome_visit_long.csv")
OUT_VISIT_WIDE_CSV  = Path("/kaggle/working/outcome_visit_wide_by_variable.csv")

# Visit order preference (anything not found falls to the end)
VISIT_PRIORITY = [
    "Visit 1", "Ramadan", "Visit 2", "Visit 3", "Visit 4 (whole Ramadan)",
    "Visit 5", "Visit 6 (Shawal)", "Visit 7"
]

# Sheets we shouldn't treat as visits
SKIP_SHEETS = {"HMC_map_patientID", "Visit_Subperiods_Spec", "Period_Bounds"}

# ========= LOAD ALL VISIT SHEETS =========
xls = pd.ExcelFile(IN_XLSX)
visit_sheets = [s for s in xls.sheet_names if s not in SKIP_SHEETS]
if not visit_sheets:
    raise RuntimeError("No visit-like sheets found. Check SKIP_SHEETS or input file.")

dfs = []
for s in visit_sheets:
    df = pd.read_excel(IN_XLSX, sheet_name=s)
    df["Visit"] = s
    dfs.append(df)

all_df = pd.concat(dfs, ignore_index=True)
# Keep original header case in the output; just trim whitespace
all_df.columns = [str(c).strip() for c in all_df.columns]

# Find a patient-id column robustly
PID_CANDIDATES = [
    "PatientID (Huawei Data)", "patientID", "patientid", "Patient ID",
    "patient id", "Code", "code"
]
pid_col = next((c for c in PID_CANDIDATES if c in all_df.columns), None)
if pid_col is None:
    lower = {c.lower(): c for c in all_df.columns}
    pid_col = next((lower[k] for k in lower if "patient" in k or k == "code"), None)
if pid_col is None:
    raise ValueError("Could not find a patient ID column. Add it to PID_CANDIDATES.")

# Visit order for choosing baseline static values
def visit_rank(v):
    return VISIT_PRIORITY.index(v) if v in VISIT_PRIORITY else len(VISIT_PRIORITY) + 99
all_df["__visit_order__"] = all_df["Visit"].map(visit_rank)

# ========= YOUR EXACT OUTPUT COLUMNS =========
VISIT_TARGETS = ["carb","meals","total_daily_dose_u","fasting_percent_29"]
STATIC_TARGETS = [
    "Age","Gender","BMI","HbA1C","Cholesterol","LDL","HDL",
    "Triglycerides","eGFR","Creatinine","Insulin_units_per_kg","SmartGuard_percent"
]

# ========= ALIAS RESOLUTION (maps messy headers → canonical names) =========
def _norm(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s)).lower().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

ALIASES = {
    # --- VISIT VARS ---
    "carb": [
        "carb","carbs","carb/day","carbs/day","carbohydrate","carbohydrates","carbohydrates per day"
    ],
    "meals": [
        "meals","meals/day","number of meals","meals per day","meal count"
    ],
    "total_daily_dose_u": [
        "total daily dose (u)","total daily dose (units)","total daily dose (unit)",
        "total daily dose","tdd","tdd (units)","tdd units","tdd u"
    ],
    "fasting_percent_29": [
        "fasting % (out of 29 days)","fasting percent (out of 29 days)",
        "fasting %","fasting percent","fasting pct","fastingpct","fasting%","fastingpercent29",
        "fasting % out of 29"
    ],

    # --- STATIC VARS ---
    "Age": ["age","age (years)","age years"],
    "Gender": ["gender","gender (m / f)","sex"],
    "BMI": ["bmi"],
    "HbA1C": ["hba1c","hba1c %","hba1c%"],
    "Cholesterol": ["cholesterol","total cholesterol"],
    "LDL": ["ldl","ldl-c","ldl cholesterol"],
    "HDL": ["hdl","hdl-c","hdl cholesterol"],
    "Triglycerides": ["triglyceride","triglycerides","tg"],
    "eGFR": ["egfr","estimated gfr","e-gfr"],
    "Creatinine": ["creatinine","serum creatinine","creat"],
    "Insulin_units_per_kg": ["insulin units/kg","insulin/kg","insulin per kg","insulinunitskg"],
    "SmartGuard_percent": ["smartguard %","smartguard%","smartguard percent","smartguardpct","smartguard"]
}

# Build normalized lookup from actual columns
norm_to_original = {}
for c in all_df.columns:
    norm_to_original.setdefault(_norm(c), c)

def resolve_one(canonical: str) -> str | None:
    # Try exact
    hit = norm_to_original.get(_norm(canonical))
    if hit: return hit
    # Try aliases
    for alias in ALIASES.get(canonical, []):
        hit = norm_to_original.get(_norm(alias))
        if hit: return hit
    return None

resolved_visit = {canon: resolve_one(canon) for canon in VISIT_TARGETS}
resolved_static = {canon: resolve_one(canon) for canon in STATIC_TARGETS}

missing_visit = [k for k, v in resolved_visit.items() if v is None]
missing_static = [k for k, v in resolved_static.items() if v is None]

print("— Column resolution —")
print("Visit vars:")
for k, v in resolved_visit.items():
    print(f"  {k:>22}  ←  {v if v else '[MISSING]'}")
print("Static vars:")
for k, v in resolved_static.items():
    print(f"  {k:>22}  ←  {v if v else '[MISSING]'}")

# ========= BUILD STATIC (one row per patient) =========
sorted_df = all_df.sort_values("__visit_order__")
static_src_cols = [v for v in resolved_static.values() if v is not None]
static_block = (
    sorted_df[[pid_col] + static_src_cols]
    .groupby(pid_col, as_index=False)
    .apply(lambda g: g.ffill().bfill().iloc[0])
    .reset_index(drop=True)
)
# Rename found columns to canonical
static_block = static_block.rename(columns={v: k for k, v in resolved_static.items() if v is not None})
# Ensure all requested static columns exist (even if missing → NaN), then order them
for c in STATIC_TARGETS:
    if c not in static_block.columns:
        static_block[c] = pd.NA
static_block = static_block[[pid_col] + STATIC_TARGETS]

# ========= BUILD VISIT (patient × visit, only your visit targets) =========
visit_src_cols = [v for v in resolved_visit.values() if v is not None]
visit_block = all_df[[pid_col, "Visit"] + visit_src_cols].copy()
visit_block = visit_block.rename(columns={v: k for k, v in resolved_visit.items() if v is not None})
# Ensure all requested visit columns exist, then order
for c in VISIT_TARGETS:
    if c not in visit_block.columns:
        visit_block[c] = pd.NA
visit_block = visit_block[[pid_col, "Visit"] + VISIT_TARGETS]

# Long + variable×visit pivots
visit_long = (
    visit_block
      .melt(id_vars=[pid_col, "Visit"], var_name="Variable", value_name="Value")
      .dropna(subset=["Value"], how="all")
)
visit_wide_by_var = (
    visit_long.pivot_table(index=[pid_col, "Variable"], columns="Visit", values="Value", aggfunc="first")
    .reset_index()
)

# ========= SAVE =========
with pd.ExcelWriter(OUT_XLSX, engine="openpyxl") as w:
    static_block.to_excel(w, index=False, sheet_name="Outcome_Static")
    visit_block.to_excel(w,  index=False, sheet_name="Outcome_By_Visit")
    visit_long.to_excel(w,   index=False, sheet_name="Outcome_By_Visit_Long")
    visit_wide_by_var.to_excel(w, index=False, sheet_name="Outcome_Var_x_Visit")

static_block.to_csv(OUT_STATIC_CSV, index=False)
visit_long.to_csv(OUT_VISIT_LONG_CSV, index=False)
visit_wide_by_var.to_csv(OUT_VISIT_WIDE_CSV, index=False)

print("\n✅ Static targets requested:", len(STATIC_TARGETS), "| found:", len([v for v in resolved_static.values() if v]))
print("   Missing static:", missing_static)
print("✅ Visit targets  requested:", len(VISIT_TARGETS),  "| found:", len([v for v in resolved_visit.values() if v]))
print("   Missing visit:", missing_visit)
print("Saved Excel →", OUT_XLSX)
print("Also CSVs   →", OUT_STATIC_CSV, OUT_VISIT_LONG_CSV, OUT_VISIT_WIDE_CSV)
